In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from collections import Counter
from pathlib import Path

In [2]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('/kaggle/input/loanstats/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# Convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

# Convert the target column values to low_risk and high_risk based on their values
df['loan_status'] = df['loan_status'].replace({
    'Current': 'low_risk',
    'Late (31-120 days)': 'high_risk',
    'Late (16-30 days)': 'high_risk',
    'Default': 'high_risk',
    'In Grace Period': 'high_risk'
})

df.reset_index(inplace=True, drop=True)

# Display the first 5 rows
print(df.head())

/tmp/ipykernel_33/3077870148.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=1)[:-2]


   loan_amnt  int_rate  installment home_ownership  annual_inc  \
0    10500.0    0.1719       375.35           RENT     66000.0   
1    25000.0    0.2000       929.09       MORTGAGE    105000.0   
2    20000.0    0.2000       529.88       MORTGAGE     56000.0   
3    10000.0    0.1640       353.55           RENT     92000.0   
4    22000.0    0.1474       520.39       MORTGAGE     52000.0   

  verification_status issue_d loan_status pymnt_plan    dti  ...  \
0     Source Verified  Mar-19    low_risk          n  27.24  ...   
1            Verified  Mar-19    low_risk          n  20.23  ...   
2            Verified  Mar-19    low_risk          n  24.26  ...   
3            Verified  Mar-19    low_risk          n  31.44  ...   
4        Not Verified  Mar-19    low_risk          n  18.76  ...   

   pct_tl_nvr_dlq  percent_bc_gt_75  pub_rec_bankruptcies  tax_liens  \
0            85.7             100.0                   0.0        0.0   
1            91.2              50.0               

In [6]:
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'next_pymnt_d', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_si

In [8]:
# Create our features
#df contains columns with all string values
training_df = pd.get_dummies(df, columns=["home_ownership","verification_status","issue_d",
                                "pymnt_plan" , "hardship_flag", "debt_settlement_flag",
                                          "initial_list_status", "next_pymnt_d", 
                                          "application_type"])

X = training_df.drop(columns="loan_status")

# Create our target
y =  df["loan_status"]

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [10]:
# Check the balance of our target values
y.value_counts()

loan_status
low_risk     68470
high_risk      347
Name: count, dtype: int64

In [11]:
# Resampling with SMOTE
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the resampled data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=1, stratify=y_resampled)


In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [13]:
# Train Balanced Random Forest
rand_forest_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rand_forest_model.fit(X_train_scaled, y_train)

# Predict and evaluate
predictions = rand_forest_model.predict(X_test_scaled)
accurate_score = accuracy_score(y_test, predictions)
print(f"Balanced Random Forest Accuracy: {accurate_score}")

# Confusion Matrix
con_mat = confusion_matrix(y_test, predictions)
con_mat_df = pd.DataFrame(con_mat, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
print(con_mat_df)

# Classification Report
print(classification_report_imbalanced(y_test, predictions))

# Feature Importances
feature_importances = rand_forest_model.feature_importances_
feature_names = X.columns
features_importance = sorted(zip(feature_names, feature_importances), key=lambda x: x[1], reverse=True)
for feature, importance in features_importance:
    print(f"{feature}: ({importance})")


/opt/conda/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/opt/conda/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/opt/conda/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(


Balanced Random Forest Accuracy: 0.9983058273696509
                  Predicted High Risk  Predicted Low Risk
Actual High Risk                17070                  47
Actual Low Risk                    11               17107
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      1.00      1.00      1.00      1.00      1.00     17117
   low_risk       1.00      1.00      1.00      1.00      1.00      1.00     17118

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     34235

issue_d_Jan-19: (0.08283377115367224)
total_rec_prncp: (0.048698120654554644)
next_pymnt_d_May-19: (0.04554648243199618)
total_pymnt_inv: (0.04219547146894464)
total_pymnt: (0.04043646922284211)
open_il_12m: (0.03833082768980802)
open_acc_6m: (0.035974976634325924)
inq_last_6mths: (0.03008025105291516)
pct_tl_nvr_dlq: (0.028472912091774348)
last_pymnt_amnt: (0.02763981415137727)
total_rec_int: (0.024314785493731995)
int_rate: (0.023448

In [14]:
# Train Easy Ensemble AdaBoost
ensemble_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ensemble_model.fit(X_train_scaled, y_train)

# Predict and evaluate
y_predict = ensemble_model.predict(X_test_scaled)
accurate_score = accuracy_score(y_test, y_predict)
print(f"Easy Ensemble AdaBoost Accuracy: {accurate_score}")

# Confusion Matrix
con_mat = confusion_matrix(y_test, y_predict)
con_mat_df = pd.DataFrame(con_mat, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"])
print(con_mat_df)

# Classification Report
print(classification_report_imbalanced(y_test, y_predict))


Easy Ensemble AdaBoost Accuracy: 0.975638965970498
                  Predicted High Risk  Predicted Low Risk
Actual High Risk                16543                 574
Actual Low Risk                   260               16858
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.98      0.97      0.98      0.98      0.98      0.95     17117
   low_risk       0.97      0.98      0.97      0.98      0.98      0.95     17118

avg / total       0.98      0.98      0.98      0.98      0.98      0.95     34235

